In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import math

# Extracting Data - Via Web Scraping

## Health Insurance Data

<img src="ans-logo.png" width="400">

### Connecting to the server

In [4]:
ans = requests.get('http://www.dados.gov.br/organization/agencia-nacional-de-saude-suplementar-ans?page=1')
ans_soup = BeautifulSoup(ans.content)
ans.status_code

200

### Getting link to files

In [5]:
# finding number of results and pages to scrap: 

results_text = ans_soup.find_all('h3')[0].text.strip()
result = re.findall('\d+', results_text)[0]

# rounding up to get number of pages: 
page_number = math.ceil(int(result)/20)
page_number

3

In [8]:
# getting the links to each files page

url = 'http://www.dados.gov.br/organization/agencia-nacional-de-saude-suplementar-ans?page='
file_links = get_file_link (url, page_number)
print(len(file_links))
file_links

47


['http://www.dados.gov.br/dataset/indice-de-reclamacoes',
 'http://www.dados.gov.br/dataset/dados-de-beneficiarios-por-regiao-geografica',
 'http://www.dados.gov.br/dataset/programa-de-qualificacao-institucional',
 'http://www.dados.gov.br/dataset/procedimentos-ambulatoriais-por-uf',
 'http://www.dados.gov.br/dataset/procedimentos-hospitalares-por-uf',
 'http://www.dados.gov.br/dataset/caderno-de-informacao',
 'http://www.dados.gov.br/dataset/http-www-ans-gov-br-perfil-do-setor-dados-abertos-dados-abertos-disponiveis-n3',
 'http://www.dados.gov.br/dataset/operadoras-e-prestadores-nao-hospitalares',
 'http://www.dados.gov.br/dataset/glossario-da-saude-suplementar',
 'http://www.dados.gov.br/dataset/produtos-e-prestadores-hospitalares',
 'http://www.dados.gov.br/dataset/caracteristicas-dos-produtos-da-saude-suplementar',
 'http://www.dados.gov.br/dataset/dados-de-pagamento-do-ressarcimento-ao-sus-por-operadora',
 'http://www.dados.gov.br/dataset/dados-de-cobranca-e-arrecadacao-do-ressarc

In [7]:
# function that get file links from all pages:

def get_file_link (url: str, page_number: int) -> list:
    '''
    This function receives:
        - a url without the page number
        - the total number of pages to go forward
    it connects to all pages available in the url
    and outputs a list of links to all files in each page.
    
    example:
    inputs: 
        url = 'http://www.dados.gov.br/organization/agencia-nacional-de-saude-suplementar-ans?page='
        page_number = 3
    output:
        list with all the complete links to access files from the pages.
        
    '''
    list_pages = []
    list_links = []
    root = 'http://' + url.split('/')[2]
    
    for i in range(1, page_number+1):
        ans = requests.get(url+str(i))
        list_pages.append (BeautifulSoup(ans.content))

    for j in range(len(list_pages)):
        soup = list_pages[j].find_all('h3', attrs={'class':'dataset-heading'})
        for k in range(len(soup)):
            link = soup[k].find_all('a')[0]['href']
            list_links.append (root + link)
   
    return list_links

From all files, filter only files related to 'plano', 'plano de saúde', 'beneficiário' or 'operadora':

In [9]:
filtered_list = [] 
pattern = ".*[Pp]lano.*|.*[Pp]lano.+[Ss]aúde.*|.*[Bb]enefici[áa]ri[oa].*|.*[Oo]peradora.*"

for file in file_links:
    file = re.findall(pattern, file)
    if len(file)>0:
        filtered_list.append(file[0])
print(len(filtered_list))
filtered_list

20


['http://www.dados.gov.br/dataset/dados-de-beneficiarios-por-regiao-geografica',
 'http://www.dados.gov.br/dataset/operadoras-e-prestadores-nao-hospitalares',
 'http://www.dados.gov.br/dataset/dados-de-pagamento-do-ressarcimento-ao-sus-por-operadora',
 'http://www.dados.gov.br/dataset/dados-gov-br-dataset-dados-de-beneficiarios-por-operadora',
 'http://www.dados.gov.br/dataset/operadoras-de-planos-privados-de-saude',
 'http://www.dados.gov.br/dataset/operadoras-de-planos-de-saude-com-registro-cancelado',
 'http://www.dados.gov.br/dataset/reajuste-de-planos-coletivos',
 'http://www.dados.gov.br/dataset/penalidades-aplicadas-a-operadoras',
 'http://www.dados.gov.br/dataset/historico-de-planos-de-saude',
 'http://www.dados.gov.br/dataset/plano-anual-de-atividades-da-auditoria-interna-paint',
 'http://www.dados.gov.br/dataset/operadoras-acreditadas',
 'http://www.dados.gov.br/dataset/informacoes-consolidadas-de-beneficiarios',
 'http://www.dados.gov.br/dataset/beneficiarios-com-vinculos-at

In [11]:
# requesting every filtered pages to access link to the file download page:

list_files = []

for page_url in filtered_list:
    soup = BeautifulSoup(requests.get(page_url).content)
    files = soup.find_all('a', attrs={'class':'heading'})

    # in case there are more than 1 files listed in the page:
    for i in range(len(files)):
        file_name = files[i].text.strip()

        if file_name.endswith('csv') or file_name.endswith('zip'):
            file_url = 'http://www.dados.gov.br' + files[i]['href']
            list_files.append(file_url)
            
        else:
            pass

### Verifiy if there are files saved in directory

### Saving new files to local directory

In [12]:
# accessing files download pages to save in local 'data' directory:

import urllib.request, urllib.parse, urllib.error 

for i in range(len(list_files)):
    new_soup = BeautifulSoup(requests.get(list_files[i]).content)
    print(new_soup.status_code)

    file = new_soup.find_all('p', attrs={'class':'muted ellipsis'})[0].find_all('a')[0]['href']
    print(file)
    
    # saving file to folder:
    file_name = file.split('/')[-1]
    print(file_name)    
    folder_name = './data'
    urllib.request.urlretrieve(file, f'{folder_name}/{file_name}')
    
# Unzipping files 
    if file_name.endswith('zip'):
        extract_zip(folder_name, file_name)
    else: 
        pass    
  

None
http://ftp.dadosabertos.ans.gov.br/FTP/PDA/dados_de_beneficiarios_por_regiao_geografica/benef_regiao_geog.zip
benef_regiao_geog.zip
None
http://ftp.dadosabertos.ans.gov.br/FTP/PDA/operadoras_e_prestadores_nao_hospitalares/operadoras_e_prestadores_nao_hospitalares.zip
operadoras_e_prestadores_nao_hospitalares.zip
None
http://ftp.dadosabertos.ans.gov.br/FTP/PDA/ressarcimento_ao_SUS_indice_efetivo_pagamento/PDA_Indice_de_Efetivo_Pagamento_Ressarcimento_SUS.csv
PDA_Indice_de_Efetivo_Pagamento_Ressarcimento_SUS.csv
None
http://ftp.dadosabertos.ans.gov.br/FTP/PDA/dados_de_beneficiarios_por_operadora/sib_ativos.zip
sib_ativos.zip
None
http://ftp.dadosabertos.ans.gov.br/FTP/PDA/dados_de_beneficiarios_por_operadora/sib_inativos.zip
sib_inativos.zip
None
http://ftp.dadosabertos.ans.gov.br/FTP/PDA/penalidades_aplicadas_a_operadoras/penalidades_aplicadas_a_operadoras.csv
penalidades_aplicadas_a_operadoras.csv
None
http://ftp.dadosabertos.ans.gov.br/FTP/PDA/historico_planos_saude/HISTORICO_PLA

IsADirectoryError: [Errno 21] Is a directory: './data/'

In [166]:
# accessory functions: 

def extract_zip(folder_name:str, zip_file:str):
    '''
    This function unzips ziped files.
    Input: folder_name or path, ziped file name
    Output: unziped files (saved in the same folder)
    '''
    
    from zipfile import ZipFile

    with ZipFile(f'{folder_name}/{zip_file}', 'r') as zipObj:
       # Extract all the contents of zip file in current directory
       zipObj.extractall(path= f'{folder_name}/')

### TESTES

In [ ]:
# teste 2 - SUCCESS

In [211]:
list_files[2]

'http://www.dados.gov.br/dataset/dados-de-pagamento-do-ressarcimento-ao-sus-por-operadora/resource/9e687209-8497-467e-8d0d-fcd310cfe782'

In [212]:
new_soup = BeautifulSoup(requests.get(list_files[2]).content)
new_soup

<!DOCTYPE html>
<!--[if IE 7]> <html lang="pt_BR" class="ie ie7"> <![endif]--><!--[if IE 8]> <html lang="pt_BR" class="ie ie8"> <![endif]--><!--[if IE 9]> <html lang="pt_BR" class="ie9"> <![endif]--><!--[if gt IE 8]><!--><html lang="pt_BR"> <!--<![endif]-->
<head>
<!--[if lte ie 8]><script type="text/javascript" src="/fanstatic/vendor/:version:2018-07-05T22:13:16/html5.min.js"></script><![endif]-->
<link href="/fanstatic/vendor/:version:2018-07-05T22:13:16/select2/select2.css" rel="stylesheet" type="text/css"/>
<link href="/fanstatic/css/:version:2018-07-05T22:13:16/main.min.css" rel="stylesheet" type="text/css"/>
<link href="/fanstatic/vendor/:version:2018-07-05T22:13:16/font-awesome/css/font-awesome.min.css" rel="stylesheet" type="text/css"/>
<!--[if ie 7]><link rel="stylesheet" type="text/css" href="/fanstatic/vendor/:version:2018-07-05T22:13:16/font-awesome/css/font-awesome-ie7.min.css" /><![endif]-->
<link href="/fanstatic/ckanext-harvest/:version:2018-07-05T22:15:46/styles/harves

In [205]:
file = new_soup.find_all('p', attrs={'class':'muted ellipsis'})[0].find_all('a')[0]['href']
file

'http://ftp.dadosabertos.ans.gov.br/FTP/PDA/ressarcimento_ao_SUS_indice_efetivo_pagamento/PDA_Indice_de_Efetivo_Pagamento_Ressarcimento_SUS.csv'

In [206]:
last_url = requests.get(file)
last_url

<Response [200]>

In [208]:
file_name = file.split('/')[-1]
file_name

'PDA_Indice_de_Efetivo_Pagamento_Ressarcimento_SUS.csv'

In [209]:
file_name = file.split('/')[-1]
print(file_name)
folder_name = './data'
urllib.request.urlretrieve(file, f'{folder_name}/{file_name}')

PDA_Indice_de_Efetivo_Pagamento_Ressarcimento_SUS.csv


('./data/PDA_Indice_de_Efetivo_Pagamento_Ressarcimento_SUS.csv',
 <http.client.HTTPMessage at 0x11ae543c8>)

In [ ]:
# teste1 - SUCCESS

In [192]:
new_soup = BeautifulSoup(requests.get(list_files[0]).content)

In [199]:
new_soup.find_all('p', attrs={'class':'muted ellipsis'})[0].find_all('a')[0]['href']

'http://ftp.dadosabertos.ans.gov.br/FTP/PDA/dados_de_beneficiarios_por_regiao_geografica/benef_regiao_geog.zip'

In [184]:
new_soup = BeautifulSoup(requests.get(list_files[0]).content)
file = new_soup.find_all('p')[0].find_all('a')[0]['href']
last_url = requests.get(file)
last_url

<Response [200]>

In [185]:
file_name = file.split('/')[-1]
print(file_name)
folder_name = './data'
urllib.request.urlretrieve(file, f'{folder_name}/{file_name}')

benef_regiao_geog.zip


('./data/benef_regiao_geog.zip', <http.client.HTTPMessage at 0x11c54f550>)

---------------

# Extracting Data - Via API Wrapper

## Google Trends API

<img src="Google.png"  width="400">

### Instaling API Wrapper

In [13]:
!pip install pytrends

Defaulting to user installation because normal site-packages is not writeable
  Created wheel for pytrends: filename=pytrends-4.7.2-py3-none-any.whl size=14261 sha256=9e429a093c8b1b2e3d310ed0ff4f51c541c63688b01a9add2b83d9079919864b
  Stored in directory: /Users/serachung/Library/Caches/pip/wheels/58/87/76/aae69ce81a925e574db418628ebbe184387c1e67320697511c
Successfully built pytrends


### Importing libraries

In [48]:
from pytrends.request import TrendReq
pytrend = TrendReq()

### Filtering related queries

In [105]:
# Finding queries related to 'plano de saúde' to use as <key words> for the search:

pytrend.build_payload(kw_list=['plano de saúde'])
related_queries = pytrend.related_queries()
related_queries

{'plano de saúde': {'top':                          query  value
  0               plano de saude    100
  1                       unimed     46
  2        unimed plano de saúde     45
  3              planos de saúde     34
  4      plano de saúde bradesco     29
  5                   plano amil     29
  6          amil plano de saúde     28
  7               bradesco saúde     28
  8                         amil     27
  9              planos de saude     21
  10       plano de saude unimed     15
  11                         ans     15
  12          lei plano de saúde     14
  13       lei de plano de saúde     14
  14     plano de saude bradesco     13
  15              bradesco saude     13
  16  plano de saúde empresarial     12
  17   plano de saúde individual     12
  18       plano de saúde preços     11
  19      hapvida plano de saúde     10
  20         plano de saude amil     10
  21     plano nacional de saúde      9
  22     reajuste plano de saúde      9
  23     carênc

In [103]:
# Finding related key words categories: 

key_word = list(related_queries.keys())[0]
list(related_queries.get(key_word).keys())

['top', 'rising']

In [98]:
# Selecting top 3 key words for the TOP category:

top_queries = list(related_queries.get('plano de saúde').values())[0]
top_queries.head()

,query,value
0,plano de saude,100
1,unimed,46
2,unimed plano de saúde,45
3,planos de saúde,34
4,plano de saúde bradesco,29


**This is the list of key words that will be queried:**

In [188]:
query_list = ['plano de saúde', top_queries.loc[0,'query'], top_queries.loc[3,'query']]

### Defining range of dates to collect data

In [186]:
# Creating a list containing all date ranges for data collection

from datetimerange import DateTimeRange
from dateutil.relativedelta import relativedelta

dates_list = []
time_range = DateTimeRange("2015-01-01", "2020-01-01")

for value in time_range.range(relativedelta(months=+1)):
    start_date = value.strftime("%Y-%m-%d")
    end_date = find_last_day(start_date)
    dates_list.append(start_date + " " + end_date)

print(len(dates_list), ' months')
dates_list

61  months


['2015-01-01 2015-01-31',
 '2015-02-01 2015-02-28',
 '2015-03-01 2015-03-31',
 '2015-04-01 2015-04-30',
 '2015-05-01 2015-05-31',
 '2015-06-01 2015-06-30',
 '2015-07-01 2015-07-31',
 '2015-08-01 2015-08-31',
 '2015-09-01 2015-09-30',
 '2015-10-01 2015-10-31',
 '2015-11-01 2015-11-30',
 '2015-12-01 2015-12-31',
 '2016-01-01 2016-01-31',
 '2016-02-01 2016-02-29',
 '2016-03-01 2016-03-31',
 '2016-04-01 2016-04-30',
 '2016-05-01 2016-05-31',
 '2016-06-01 2016-06-30',
 '2016-07-01 2016-07-31',
 '2016-08-01 2016-08-31',
 '2016-09-01 2016-09-30',
 '2016-10-01 2016-10-31',
 '2016-11-01 2016-11-30',
 '2016-12-01 2016-12-31',
 '2017-01-01 2017-01-31',
 '2017-02-01 2017-02-28',
 '2017-03-01 2017-03-31',
 '2017-04-01 2017-04-30',
 '2017-05-01 2017-05-31',
 '2017-06-01 2017-06-30',
 '2017-07-01 2017-07-31',
 '2017-08-01 2017-08-31',
 '2017-09-01 2017-09-30',
 '2017-10-01 2017-10-31',
 '2017-11-01 2017-11-30',
 '2017-12-01 2017-12-31',
 '2018-01-01 2018-01-31',
 '2018-02-01 2018-02-28',
 '2018-03-01

`This is a side function`

In [182]:
def find_last_day(date):
    
    '''
    This function receives any date and returns the last day of the informed month.
    example: 
        input: 2015-01-02
        output: 2015-01-31
    '''
    
    year = date.split('-')[0]
    month = date.split('-')[1]
    day = calendar.monthrange(int(year), int(month))[1]
    
    return year + '-' + month + '-' + str(day)

### Search queries made in Brazil 
- Based on main related key words
- Monthly, since January 2015

In [198]:
# Building Payload for all dates:

list_df = []

for date_range in dates_list:

    pytrend.build_payload(kw_list = query_list, 
                          geo = 'BR',
                          timeframe = date_range)
    
    df = pytrend.interest_by_region(inc_geo_code=True, inc_low_vol=True)
    df['date'] = date_range
    list_df.append(df)

full_df = pd.concat(list_df)
full_df

,geoCode,plano de saúde,plano de saude,planos de saúde,date
geoName,,,,,
Federal District,BR-DF,23,70,7,2015-01-01 2015-01-31
State of Acre,BR-AC,0,0,0,2015-01-01 2015-01-31
State of Alagoas,BR-AL,24,69,7,2015-01-01 2015-01-31
State of Amapá,BR-AP,42,58,0,2015-01-01 2015-01-31
State of Amazonas,BR-AM,20,66,14,2015-01-01 2015-01-31
...,...,...,...,...,...
State of Roraima,BR-RR,42,58,0,2020-01-01 2020-01-31
State of Santa Catarina,BR-SC,22,72,6,2020-01-01 2020-01-31
State of Sergipe,BR-SE,22,72,6,2020-01-01 2020-01-31


----------------

# Saving to DataBase

## Connecting do Postgres

In [200]:
import sqlalchemy as db

In [201]:
db_server = 'postgresql'
user = 'postgres'
password = XXX
ip = 'localhost'
db_name = 'Project_03'

# create the engine
engine = db.create_engine(f'{db_server}://{user}:{password}@{ip}/{db_name}')

# localhost = 127.0.0.1

# open the connection
conn = engine.connect()

## Saving ANS Web scraped data to DB

In [204]:
# code here

## Saving Google Trends information to DB

In [202]:
full_df.to_sql('google_trends', con=conn, if_exists='replace', index=False)

------------

# Sources

## For Google Trends API

https://pypi.org/project/pytrends/

https://github.com/Tanu-N-Prabhu/Python/blob/master/Google_Trends_API.ipynb

## Additional Data Sources to access

### Social Data

In [2]:
# IBGE (Pesquisa de Orçamentos Familiares/ Censo Demográfico)
'http://www.dados.gov.br/dataset?tags=Consumo&organization=instituto-brasileiro-de-geografia-e-estatistica-ibge&_tags_limit=0'

'http://www.dados.gov.br/dataset?tags=Consumo&organization=instituto-brasileiro-de-geografia-e-estatistica-ibge&_tags_limit=0'

In [3]:
# MINISTÉRIO DA SAÚDE (Cadastro Nacional de Estabelecimentos de Saúde - CNES)
'http://www.dados.gov.br/organization/5b283f30-ced3-4ccc-b44a-406e8a92e1ad?groups=dados-em-destaque'

'http://www.dados.gov.br/organization/5b283f30-ced3-4ccc-b44a-406e8a92e1ad?groups=dados-em-destaque'

### EPIDEMIAS / PANDEMIAS

In [205]:
# GHO
'https://apps.who.int/gho/data/node.country.country-BRA?lang=en'

'https://apps.who.int/gho/data/node.country.country-BRA?lang=en'

In [ ]:
# Pandemia 2003 -2004 – Gripe Aviária


In [ ]:
# Pandemia de 2009 – Gripe A (H1N1)  pdm 09 – Gripe Suína.